In [15]:
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point,Polygon
import csv
import fiona
import random
from geopandas.tools import sjoin
import time
from prettytable import PrettyTable
from geopandas import GeoDataFrame, read_file 
import geoplot


ImportError: cannot import name 'drivers' from 'fiona' (unknown location)

Defining input random coordinates

Sample Data Points =1000,can be increased by changing the Global Variable

In [ ]:
lat = []
lon = []
data_points = 1000
for i in range(data_points):
    x = random.uniform(-90,90)
    lat.append(x)
    
    
for i in range(data_points):
    x = random.uniform(-180,180)
    lon.append(x)

df_cord = pd.DataFrame(lon , columns = ['Lon'])
df_cord['Lat'] = lat
geometry = [Point(xy) for xy in zip(df_cord.Lon, df_cord.Lat)]
df_cord = df_cord.drop(['Lon' , 'Lat' ], axis=1)
crs = {'init': 'epsg:4326'}
gdf = geopandas.GeoDataFrame(df_cord, crs=crs, geometry=geometry)


In [ ]:
print(gdf.head)

Visualizing the input coordinates on World Map

In [ ]:
plt.ion()
world_boundary = geopandas.read_file("wc_v3__uuid.shp")
world_boundary.plot( figsize = (40,30))


In [ ]:
import shapefile
from json import dumps

   # read the shapefile
reader = shapefile.Reader("wc_v3__uuid.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
  atr = dict(zip(field_names, sr.record))
  geom = sr.shape.__geo_interface__
  buffer.append(dict(type="Feature", \
    geometry=geom, properties=atr)) 

   # write the GeoJSON file

geojson = open("pyshp-demo.json", "w")
geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
geojson.close()

In [ ]:
plt.ion()
eez_boundary = geopandas.read_file("eez_v3__uuid.shp")
eez_boundary.plot( figsize = (40,30))


In [ ]:
print(eez_boundary.head())
print(world_boundary.head())

In [ ]:
print(eez_boundary.shape)
print(world_boundary.shape)


In [ ]:
# world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
ax = world_boundary.plot( edgecolor='black' ,figsize = (40,30))


gdf.plot(ax=ax, color='red')
plt.title("World map showing location of given coordinate")
print(world_boundary.head(3))
plt.show()

In [ ]:
# world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
ax = eez_boundary.plot( edgecolor='black' ,figsize = (40,30))


gdf.plot(ax=ax, color='red')
plt.title("World map showing location of given coordinate")
print(eez_boundary.head(3))
plt.show()

In [ ]:
print(world_boundary.shape)

In [ ]:
gdf1 = pd.concat([world_boundary, eez_boundary])
print(type(gdf1))
print(gdf1.shape)
# gdf1.plot( figsize = (40,30))

Custom Module to check if a given coordinate lies within the boundaries of a region

In [ ]:
# shapes = world_boundary.merge(eez_boundary, on='uuid_0')
# res_intersection = geopandas.overlay(world_boundary, eez_boundary, how='intersection')


In [ ]:
polys = geopandas.GeoSeries(world.loc[world['name'] == 'Russia'].geometry).values[0]
_pnts = Point(77.580643 , 12.972442)
_pnts1 = Point(55.018803 , 0.1278)
print("Is London present inside polygon(Russia) : " ,_pnts1.within(polys))
print("Is Bangalore present inside polygon(Russia) : " ,_pnts.within(polys))



In [ ]:
df_new = world[['continent' , 'name' , 'geometry']]

In [ ]:
# df_india = pd.DataFrame(sjoin(gdf,df_new[df_new.name == "India"], how = 'left'))
# print(df_india.dropna())
# working 

In [ ]:
# df_india = pd.DataFrame(sjoin(gdf,df_new[df_new.name == "India"], how = 'left'))
# df_russia = pd.DataFrame(sjoin(gdf,df_new[df_new.name == "Russia"], how = 'left'))
# df_merge = df_india.fillna(df_russia)
# print(df_india.columns)

Running the above module for all the countries and returning a single output file that states to which country the coordinate belongs

In [ ]:
start_time = time.time()  

df_merged = pd.DataFrame(np.nan, index=range(data_points), columns=['name'])
for country in df_new.name:
    df_country = pd.DataFrame(sjoin(gdf, df_new[df_new.name == country], how='left'))
    df_merged = df_merged.fillna(df_country)
    
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print(df_merged.shape)

In [ ]:
print(df_merged)

In [ ]:
print("Without Multiprocessing")
x = PrettyTable()

x.field_names = ["S.No." ," Number of Points", "Time taken to run"]
x.add_row([0, 1000 ,"13 seconds"])
x.add_row([0, 2000 ,"22 seconds"])
x.add_row([0, 4000 ,"44 seconds"])
x.add_row([0, 8000 ,"94 seconds"])


print(x)
print("Without Multiprocessing")
x = PrettyTable()

x.field_names = ["S.No." ," Number of Points", "Time taken to run"]
x.add_row([0, 1000 ,"13 seconds"])
x.add_row([0, 10000 ,"107 seconds"])
x.add_row([0, 100000 ,"18 minutes"])




print(x)

In [ ]:
print("Without Multiprocessing")
x = PrettyTable()

x.field_names = ["S.No." ," Number of Points", "Time taken to run"]
x.add_row([0, 1000 ,"13 seconds"])
x.add_row([0, 10000 ,"107 seconds"])
x.add_row([0, 100000 ,"18 minutes"])




print(x)

In [ ]:
import multiprocessing

multiprocessing.cpu_count()